In [1]:
import sys
sys.path.append('/home/vinnylg/projects/covid19datascience')
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels

In [4]:
read_cc = True #and False
read_tb = True #and False

In [5]:
cc = CasosConfirmados()
if read_cc:
    cc.read('/home/vinnylg/projects/covid19datascience/input/Casos confirmados PR.xlsx')
    cc.save()
else:
    cc.load()

reading Casos Confirmados time elepsed:00:03:37
saving Casos Confirmados time elepsed:00:00:05


In [6]:
casosc = cc.get_casos()
del casosc['identificacao']
print(f"casosc len: {len(casosc)}\n")
print(casosc.columns.values)

casosc len: 970957

['ordem' 'ibge_res_pr' 'rs_res_pr' 'nome' 'sexo' 'idade' 'mun_resid'
 'mun_atend' 'rs' 'laboratorio' 'dt_diag' 'comunicacao' 'is' 'obito'
 'data_obito' 'excluir' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [7]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
print(casost.columns.values)

Reading TbPacientes time elepsed:00:04:15
casost len: 970957

['identificacao' 'ibge_res_pr' 'ibge_atend_pr' 'nome' 'sexo' 'idade'
 'mun_resid' 'mun_atend' 'laboratorio' 'dt_diag' 'dt_notificacao'
 'dt_inicio_sintomas' '_dia_isolamento' 'dt_viagem_retorno' 'local_viagem'
 'exposicao' 'data_de_internamento' 'local' 'clinico_uti' 'dt_alta'
 'obito' 'dt_obito' 'muni_res_pr' 'hist_viagem' 'casos_novos' 'telefone'
 'dt_atualizacao' 'fonte' 'status' 'eas' 'privado_sus' 'uti'
 'dt_internamento' 'evolucao' 'dt_evolucao' 'temp_permanencia'
 'cod_laboratorio' 'idade_original' 'dt_com_obito' 'dt_com_recuperado'
 'idade_mais' 'idade_menos' 'hash' 'hash_less' 'hash_more' 'hash_atend'
 'hash_less_atend' 'hash_more_atend' 'hash_diag' 'hash_obito']


In [8]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [10]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

23489
947468


In [13]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

969989
1585


In [14]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

971574
1


In [15]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

971574
1


In [16]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

971575
0


,ordem,ibge_res_pr,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [17]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [18]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag']==ordem_duplicado['hash_diag'])

741 741


True

In [19]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]

In [20]:
print(len(casosct))
casosct = casosct.drop(index=casosct.loc[casosct.duplicated('identificacao',keep='last')].index)
print(len(casosct))

971575
970957


In [21]:
print(casosct.columns)
print(len(casosct))

Index(['ordem', 'ibge_res_pr', 'rs_res_pr', 'nome', 'sexo', 'idade',
       'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'dt_diag', 'comunicacao',
       'is', 'obito', 'data_obito', 'excluir', 'hash', 'hash_less',
       'hash_more', 'hash_atend', 'hash_less_atend_x', 'hash_more_atend_x',
       'hash_diag', 'hash_obito', 'identificacao', 'ibge_atend_pr',
       'dt_com_obito', 'status', 'dt_com_recuperado', 'hash_less_atend_y',
       'hash_more_atend_y'],
      dtype='object')
970957


In [22]:
len(casosct.loc[casosct['obito']=='SIM'])

23493

In [23]:
casosct = casosct.sort_values(['comunicacao','nome']).copy()
casosct['ordem'] = [ x for x in range(1,len(casosct)+1)]
casosct['uf_resid'] = 'PR'
casosct.loc[casosct['rs']==99,'uf_resid'] = 'FORA'

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct = casosct[['ordem', 'identificacao','uf_resid','ibge_res_pr','ibge_atend_pr', 'rs', 'nome', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'laboratorio', 'dt_diag', 'comunicacao', 'is','obito', 'data_obito', 'dt_com_obito', 'status', 'dt_com_recuperado', 'evolucao', 'data_evolucao', 'data_com_evolucao', 'hash', 'hash_less', 'hash_more', 'hash_atend', 'hash_diag', 'hash_obito']]

cc.save(casosct)
casosct

,ordem,identificacao,uf_resid,ibge_res_pr,ibge_atend_pr,rs,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,comunicacao,is,obito,data_obito,dt_com_obito,status,dt_com_recuperado,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_diag,hash_obito
23500,1,19775,PR,410690,410690,2,LUIZ ANTONIO CAXAMBU FREITAS,M,54,CURITIBA,CURITIBA,GENOPRIMER,2020-03-11,2020-03-11,2020-03-09,NAO,NaT,NaT,Recuperado,NaT,1,NaT,NaT,LUIZANTONIOCAXAMBUFREITAS54CURITIBA,LUIZANTONIOCAXAMBUFREITAS53CURITIBA,LUIZANTONIOCAXAMBUFREITAS55CURITIBA,LUIZANTONIOCAXAMBUFREITAS54CURITIBA,LUIZANTONIOCAXAMBUFREITAS11032020,NaN
23502,2,22343,PR,410690,410690,2,MARIANA CAXAMBU FREITAS,F,25,CURITIBA,CURITIBA,GENOPRIMER,2020-03-11,2020-03-11,2020-03-01,NAO,NaT,NaT,Recuperado,NaT,1,NaT,NaT,MARIANACAXAMBUFREITAS25CURITIBA,MARIANACAXAMBUFREITAS24CURITIBA,MARIANACAXAMBUFREITAS26CURITIBA,MARIANACAXAMBUFREITAS25CURITIBA,MARIANACAXAMBUFREITAS11032020,NaN
23489,3,8838,PR,410550,410550,13,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaT,NAO,NaT,NaT,Recuperado,NaT,1,NaT,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
23490,4,10932,PR,410690,410690,2,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07,NAO,NaT,NaT,Recuperado,NaT,1,NaT,NaT,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
23491,5,10933,PR,410690,410690,2,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08,NAO,NaT,NaT,Recuperado,NaT,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969984,970953,1056002,PR,411820,411820,1,ZULEIDE APARECIDA MARIA,F,62,PARANAGUA,PARANAGUA,"COVID-19, Biologia Molecular",2021-01-22,2021-05-06,NaT,NAO,NaT,NaT,NaN,NaT,3,NaT,NaT,ZULEIDEAPARECIDAMARIA62PARANAGUA,ZULEIDEAPARECIDAMARIA61PARANAGUA,ZULEIDEAPARECIDAMARIA63PARANAGUA,ZULEIDEAPARECIDAMARIA62PARANAGUA,ZULEIDEAPARECIDAMARIA22012021,NaN
969985,970954,1056003,PR,410550,410550,13,ZULEIDE DIAS OLIVEIRA,F,54,CIANORTE,CIANORTE,Teste Rápido Antígeno (Imunocromatografia),2021-05-05,2021-05-06,2021-04-30,NAO,NaT,NaT,NaN,NaT,3,NaT,NaT,ZULEIDEDIASOLIVEIRA54CIANORTE,ZULEIDEDIASOLIVEIRA53CIANORTE,ZULEIDEDIASOLIVEIRA55CIANORTE,ZULEIDEDIASOLIVEIRA54CIANORTE,ZULEIDEDIASOLIVEIRA05052021,NaN
969986,970955,1056004,PR,411120,411120,7,ZULEIDE TERESINHA PICCOLI,F,68,ITAPEJARA D'OESTE,ITAPEJARA D'OESTE,Teste Rápido Antígeno (Imunocromatografia),2021-05-05,2021-05-06,2021-05-03,NAO,NaT,NaT,NaN,NaT,3,NaT,NaT,ZULEIDETERESINHAPICCOLI68ITAPEJARADOESTE,ZULEIDETERESINHAPICCOLI67ITAPEJARADOESTE,ZULEIDETERESINHAPICCOLI69ITAPEJARADOESTE,ZULEIDETERESINHAPICCOLI68ITAPEJARADOESTE,ZULEIDETERESINHAPICCOLI05052021,NaN
969987,970956,1056005,PR,410765,410765,2,ZULMIRO PRIMO BIAZUS,M,74,FAZENDA RIO GRANDE,FAZENDA RIO GRANDE,"COVID-19, Biologia Molecular",2021-05-05,2021-05-06,2021-04-25,NAO,NaT,NaT,NaN,NaT,3,NaT,NaT,ZULMIROPRIMOBIAZUS74FAZENDARIOGRANDE,ZULMIROPRIMOBIAZUS73FAZENDARIOGRANDE,ZULMIROPRIMOBIAZUS75FAZENDARIOGRANDE,ZULMIROPRIMOBIAZUS74FAZENDARIOGRANDE,ZULMIROPRIMOBIAZUS05052021,NaN


In [25]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_resid'])[['ordem']].count().unstack().droplevel(0,1)[['PR','FORA']]
total['TOTAL'] = total.sum(1)
total.astype(float)

uf_resid,PR,FORA,TOTAL
evolucao,,,
1,691.112,2.137,693.249
2,23.346,147,23.493
3,250.749,3.466,254.215


In [26]:
casosct.loc[casosct['rs']==99,'municipio_pr'] = 'FORA'
casosct.loc[casosct['rs']!=99,'municipio_pr'] = casosct.loc[casosct['rs']!=99,'mun_resid']
#merge macro/rs_nome
total_municipios = casosct.groupby(['ibge_res_pr','rs','municipio_pr','evolucao'])[['ordem']].count().unstack()
total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.append(pd.Series(total_municipios.sum(0),name=('','','TOTAL GERAL')))
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]
total_municipios.astype(float)

,,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
ibge_res_pr,rs,municipio_pr,,,,
410010,18,ABATIA,622,10,503,109
410020,2,ADRIANOPOLIS,1.072,21,971,80
410030,2,AGUDOS DO SUL,474,16,276,182
410040,2,ALMIRANTE TAMANDARE,7.837,298,1.693,5.846
410045,11,ALTAMIRA DO PARANA,136,4,105,27
...,...,...,...,...,...,...
412865,5,VIRMOND,199,3,13,183
412870,7,VITORINO,588,17,553,18
412880,12,XAMBRE,261,5,207,49


In [27]:
casosct['mes_comunicacao'] = casosct['comunicacao'].apply(lambda x: f"{x.year}_{str(x.month).zfill(2)}")
casosct.groupby('mes_comunicacao')[['ordem']].count().astype(float)

,ordem
mes_comunicacao,
2020_03,182
2020_04,1.229
2020_05,3.304
2020_06,17.973
2020_07,53.120
2020_08,55.639
2020_09,46.924
2020_10,32.993
2020_11,65.930


In [28]:
#for macro:

writer = pd.ExcelWriter(join("output","novos_casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_municipios.to_excel(writer,'total_municipios')
casosct.to_excel(writer,'casos')

for mes, df in casosct.groupby('mes_comunicacao'):
    df.to_excel(writer,sheet_name=mes,index=None)

writer.save()